# Retrieving Qurtubah's Venues 

this notebook shows the process of retrieving data from Foursquare API and some of the data cleaning done in the process. 

First I was going to scrape data from Riyadh Guide, but then I dicovered that the data is out dated, it was last updated in 2016 

In [1]:
import requests
import lxml.html as lh
import pandas as pd

this method is used to calculate the distance in kms from the origin ( the center of the neiborhood in our case ) to the disenation point given the lat and lng

In [373]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d



In [2]:
url='https://www.alriyadh.gov.sa/ar/Pages/RiyadhGuideAs.aspx?page'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
[len(T) for T in tr_elements[:10]]


[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [7]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (name)
    col.append((name,[]))

الاسم
نوع المكان
الشارع
الحي
البلدية


In [10]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=5:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [11]:
[len(C) for (title,C) in col]

[10, 10, 10, 10, 10]

In [12]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [13]:
df.head()

,الاسم,نوع المكان,الشارع,الحي,البلدية
0,المدرسة الإبتدائية التاسعة والعشرون بنات,مبنى تعليمي,ابراهيم الشامي,البطيحا,البطحاء
1,حديقة جبرة,حديقة,علي بن المنير,البطيحا,البطحاء
2,الثانوية الثانية بنات,مبنى تعليمي,علي بن المنير,البطيحا,البطحاء
3,مؤسسة سليمان حمد السيف _ نقليات السيف _ نقل وت...,مبنى حكومي,,الخالدية,البطحاء
4,مسجد جمعة بحي الخالدية,مسجد,,الخالدية,البطحاء


In [31]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains


driver =webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.alriyadh.gov.sa/ar/Pages/RiyadhGuideAs.aspx?page")
button = driver.find_element_by_css_selector('.InvestementsCompetVisual .GridViewLink, .GridView .table tr:last-child a')
button.click()
print( driver.find_element_by_css_selector('.InvestementsCompetVisual .GridViewLink, .GridView .table tr:last-child a').text)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [/Users/workplace/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


1


# Getting Data From FourSquar API 

I managed to scrape most data of amennites from google maps using Apify with thier free credit, then I had to scrape the rest using octoparse but it wasn't very efficient, so I diceded to use foursquare API instead.
## Clothing stores data

In [58]:
import json, requests
import pandas as pd
import folium
from geopy.geocoders import Nominatim 


In [194]:

CLIENT_ID= 'M2LR3AYXV3Y14T4H2LKKX4WIWLNH3QD4F20E32EP3FCNQ5YF'
CLIENT_SECRET= 'UFYEBOCXOILJ234FH1WQK3Y12PLLX4RFPW0VE134VYRE0R2A'
VERSION='20180323'
neighborhood_latitude='24.8166' 
neighborhood_longitude= '46.7320'
query="Apparel"
radius=1000
LIMIT=200

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    query,
    radius, 
    LIMIT)
results = requests.get(url).json()
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [195]:
data

{'meta': {'code': 200, 'requestId': '6193dbbad70c677952dcb023'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Riyadh',
  'headerFullLocation': 'Riyadh',
  'headerLocationGranularity': 'city',
  'query': 'clothing',
  'totalResults': 144,
  'suggestedBounds': {'ne': {'lat': 24.9165990999991,
    'lng': 46.84196721181082},
   'sw': {'lat': 24.716600900000902, 'lng': 46.62203278818918}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '538b1953498ed1082715f769',
       'name': 'Nike Store',
       'contact': {},
       'location': {'address': 'Prince Saud bin Mohammed St.',
        'lat': 24.81399259983418,
        'lng': 46.72528907033608,
        'labeledLatLngs': [{'label': 'disp

In [196]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.formattedAddress']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
#nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng,formattedAddress
0,Nike Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",24.813993,46.725289,"[Prince Saud bin Mohammed St., الرياض, المملكة..."
1,Tommy Hilfiger,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",24.814086,46.725556,"[الرياض 13251, المملكة العربية السعودية]"
2,Sport Podium (منصة الرياضة),"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",24.809469,46.734692,"[Saeed Bin Zaid St, الرياض, المملكة العربية ال..."
3,SportsDirect (سبورتس دايركت),"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",24.809920,46.734541,[المملكة العربية السعودية]
4,H&M (اتش اند ام),"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",24.809493,46.734518,"[Saeed bin zaid St (Saeed bin zaid St), الرياض..."


In [197]:
nearby_venues.shape

(8, 5)

In [198]:
nearby_venues.categories[0][0]

{'id': '4bf58dd8d48988d103951735',
 'name': 'Clothing Store',
 'pluralName': 'Clothing Stores',
 'shortName': 'Apparel',
 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_',
  'suffix': '.png'},
 'primary': True}

In [199]:
nearby_venues.categories[0][0]['shortName']

'Apparel'

In [200]:
len(nearby_venues.categories)

8

In [201]:
i,j=0,0
for i in range(len(nearby_venues.categories)):
    for j in range(len(nearby_venues.categories)):
        nearby_venues.categories[i]=nearby_venues.categories[i][j]['shortName']
        i+=1
        j+=1
        break

<ipython-input-201-84c240ceeb5c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_venues.categories[i]=nearby_venues.categories[i][j]['shortName']


In [202]:
nearby_venues.dtypes

name                 object
categories           object
lat                 float64
lng                 float64
formattedAddress     object
dtype: object

In [209]:
nearby_venues

,name,categories,lat,lng,formattedAddress
0,Nike Store,Apparel,24.813993,46.725289,"[Prince Saud bin Mohammed St., الرياض, المملكة..."
1,Tommy Hilfiger,Apparel,24.814086,46.725556,"[الرياض 13251, المملكة العربية السعودية]"
2,Sport Podium (منصة الرياضة),Apparel,24.809469,46.734692,"[Saeed Bin Zaid St, الرياض, المملكة العربية ال..."
3,SportsDirect (سبورتس دايركت),Apparel,24.809920,46.734541,[المملكة العربية السعودية]
4,H&M (اتش اند ام),Apparel,24.809493,46.734518,"[Saeed bin zaid St (Saeed bin zaid St), الرياض..."
5,Foot Locker (فوت لوكر),Shoes,24.809584,46.734592,"[Al Nakhlah Plaza, الرياض 13244, المملكة العرب..."
6,Skechers (سكيتشرز),Shoes,24.814124,46.725047,"[الرياض, المملكة العربية السعودية]"
7,Payless ShoeSource,Shoes,24.809167,46.735121,[المملكة العربية السعودية]


In [205]:
nearby_venues['categories'].unique()

array(['Apparel', 'Shoes'], dtype=object)

In [215]:
nearby_venues['formattedAddress']='Qurtubah'

In [216]:
nearby_venues

,name,categories,lat,lng,formattedAddress
0,Nike Store,Apparel,24.813993,46.725289,Qurtubah
1,Tommy Hilfiger,Apparel,24.814086,46.725556,Qurtubah
2,Sport Podium (منصة الرياضة),Apparel,24.809469,46.734692,Qurtubah
3,SportsDirect (سبورتس دايركت),Apparel,24.809920,46.734541,Qurtubah
4,H&M (اتش اند ام),Apparel,24.809493,46.734518,Qurtubah
5,Foot Locker (فوت لوكر),Shoes,24.809584,46.734592,Qurtubah
6,Skechers (سكيتشرز),Shoes,24.814124,46.725047,Qurtubah
7,Payless ShoeSource,Shoes,24.809167,46.735121,Qurtubah


In [217]:
nearby_venues.to_csv('clothing.csv', sep ='\t')

## Requesting Park Data 

In [263]:
query="Park"
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    query,
    radius, 
    LIMIT)
results = requests.get(url).json()
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [264]:
data['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '573e2038498e8009a78af051',
   'name': 'Qurtubah Walking Area',
   'contact': {},
   'location': {'address': 'Qurtubah',
    'crossStreet': 'Radwah street',
    'lat': 24.802411,
    'lng': 46.734318,
    'labeledLatLngs': [{'label': 'display',
      'lat': 24.802411,
      'lng': 46.734318}],
    'distance': 1596,
    'postalCode': '11187',
    'cc': 'SA',
    'city': 'Riyadh',
    'state': 'Central',
    'country': 'المملكة العربية السعودية',
    'formattedAddress': ['Qurtubah (Radwah street)',
     'Riyadh 11187',
     'المملكة العربية السعودية']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'verified

In [314]:
parks = results['response']['groups'][0]['items']
parks=data['response']['groups'][0]['items']
nearby_parks = pd.json_normalize(parks)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.formattedAddress']
nearby_parks =nearby_parks.loc[:, filtered_columns]
nearby_parks.columns = [col.split(".")[-1] for col in nearby_parks.columns]
nearby_parks.head()



,name,categories,lat,lng,formattedAddress
0,Qurtubah Walking Area,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",24.802411,46.734318,"[Qurtubah (Radwah street), Riyadh 11187, الممل..."
1,الفلاح بارك,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",24.799951,46.699933,"[الرياض, المملكة العربية السعودية]"
2,حديقة الفلاح,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",24.792016,46.706461,"[الرياض, المملكة العربية السعودية]"
3,Qurtubah Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",24.805671,46.740503,[المملكة العربية السعودية]
4,Alwaha Garden (حديقة الواحة),"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",24.736278,46.720105,"[Abha St., الرياض 12442, المملكة العربية السعو..."


In [315]:
i,j=0,0
for i in range(len(nearby_parks.categories)):
    for j in range(len(nearby_parks.categories)):
        nearby_parks.categories[i]=nearby_parks.categories[i][j]['shortName']
        i+=1
        j+=1
        break

<ipython-input-315-5d32c6c9fa48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_parks.categories[i]=nearby_parks.categories[i][j]['shortName']


In [316]:
nearby_parks['categories'].unique()

array(['Park', 'Trail', 'Sculpture', 'Track', 'Plaza', 'Theme Park',
       'Event Space', 'Campground'], dtype=object)

In [317]:
nearby_parks_only=nearby_parks[nearby_parks['categories']=='Park'].head()

In [318]:
nearby_parks_only.shape

(5, 5)

In [319]:
nearby_parks_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              5 non-null      object 
 1   categories        5 non-null      object 
 2   lat               5 non-null      float64
 3   lng               5 non-null      float64
 4   formattedAddress  5 non-null      object 
dtypes: float64(2), object(3)
memory usage: 240.0+ bytes


In [320]:
nearby_parks_only['formattedAddress'] = nearby_parks['formattedAddress'].astype(str)

In [337]:
neiborhood= ['قرطبة','Qurtubah','qurtubah','قرطبه']

In [340]:
nearby_parks_only=nearby_parks_only[nearby_parks_only.name.str.contains('|'.join(neiborhood)) | nearby_parks_only.formattedAddress.str.contains('|'.join(neiborhood)) ]

In [341]:
nearby_parks_only.to_csv('parks.csv', sep ='\t')

## Requesting Bank & ATM Data 

In [342]:
query="ATM"
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    query,
    radius, 
    LIMIT)
results = requests.get(url).json()
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [344]:
atm = results['response']['groups'][0]['items']
atm=data['response']['groups'][0]['items']
nearby_atm = pd.json_normalize(atm)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.formattedAddress']
nearby_atm =nearby_atm.loc[:, filtered_columns]
nearby_atm.columns = [col.split(".")[-1] for col in nearby_atm.columns]
nearby_atm.head()



,name,categories,lat,lng,formattedAddress
0,SABB ATM (صراف ساب),"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",24.804728,46.749004,"[Dammam Road, الرياض, المملكة العربية السعودية]"
1,البنك الاهلي,"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",24.824568,46.772067,"[المونسيه (شارع الصحابه), الرياض 13253, المملك..."
2,Alinma Bank (مصرف الإنماء),"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",24.846634,46.782362,"[الرياض, المملكة العربية السعودية]"
3,SABB Bank ATM (بنك ساب),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",24.791585,46.729553,"[Dammam Road (Al Hamra), الرياض, المملكة العرب..."
4,Alinma Bank (مصرف الإنماء),"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",24.765788,46.706653,"[Al Imam Saud Bin Abdulaziz Rd, الرياض, المملك..."


In [346]:
i,j=0,0
for i in range(len(nearby_atm.categories)):
    for j in range(len(nearby_atm.categories)):
        nearby_atm.categories[i]=nearby_atm.categories[i][j]['shortName']
        i+=1
        j+=1
        break

<ipython-input-346-42a68b69bd37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_atm.categories[i]=nearby_atm.categories[i][j]['shortName']


In [348]:
nearby_atm['categories'].unique()

array(['ATM', 'Bank', 'Gas Station', 'Grocery Store'], dtype=object)

In [367]:
nearby_atm_only=nearby_atm[nearby_atm['categories'].isin(['ATM','Bank'])]

In [368]:
nearby_atm_only['categories'].unique()

array(['ATM', 'Bank'], dtype=object)

In [369]:
nearby_atm_only.shape

(61, 5)

In [370]:
nearby_atm_only['formattedAddress'] = nearby_atm_only['formattedAddress'].astype(str)

<ipython-input-370-2e40377512dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_atm_only['formattedAddress'] = nearby_atm_only['formattedAddress'].astype(str)


In [372]:
nearby_atm_only

,name,categories,lat,lng,formattedAddress
0,SABB ATM (صراف ساب),ATM,24.804728,46.749004,"['Dammam Road', 'الرياض', 'المملكة العربية الس..."
1,البنك الاهلي,ATM,24.824568,46.772067,"['المونسيه (شارع الصحابه)', 'الرياض 13253', 'ا..."
2,Alinma Bank (مصرف الإنماء),ATM,24.846634,46.782362,"['الرياض', 'المملكة العربية السعودية']"
3,SABB Bank ATM (بنك ساب),Bank,24.791585,46.729553,"['Dammam Road (Al Hamra)', 'الرياض', 'المملكة ..."
4,Alinma Bank (مصرف الإنماء),ATM,24.765788,46.706653,"['Al Imam Saud Bin Abdulaziz Rd', 'الرياض', 'ا..."
...,...,...,...,...,...
58,Car o2 gas station,ATM,24.789121,46.648067,"['Alghatghay Street (Before Rabie Plaza)', 'ال..."
59,HSBC Bank,ATM,24.758399,46.653206,"['North Olaya Street', 'الرياض', 'المملكة العر..."
60,SABB Bank ATM (بنك ساب),Bank,24.738949,46.682732,"['King AbdulAziz Street (Mursalat)', 'الرياض',..."
62,Al Rajhi ATMs (صرافات الراجحي),ATM,24.747950,46.654474,"['King Fahad Road (Murooj District)', 'الرياض'..."


In [383]:
nearby_atm_only['lng'] = nearby_atm_only['lng'].astype(float)
nearby_atm_only['lat'] = nearby_atm_only['lat'].astype(float)

<ipython-input-383-e87c424fd236>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_atm_only['lng'] = nearby_atm_only['lng'].astype(float)
<ipython-input-383-e87c424fd236>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_atm_only['lat'] = nearby_atm_only['lat'].astype(float)


### Solving the problem of inconsisstant addresses 
using street/dist name wasn't convenient as a filtreing method due to having multible formats and languages in the 'formattedadress' column, so I used the previously disscused function to create a new column that includes the distance from Qurtubah to a certain venue.

In [379]:
distance((float(neighborhood_latitude),float(neighborhood_longitude)),(24.804728,46.749004))

2.1652162239257975

In [399]:
nearby_atm_only['distance'] = nearby_atm_only.apply(lambda row: distance((float(neighborhood_latitude),float(neighborhood_longitude)),(row['lat'],row['lng'])), axis=1)

<ipython-input-399-67f8fa256c10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_atm_only['distance'] = nearby_atm_only.apply(lambda row: distance((float(neighborhood_latitude),float(neighborhood_longitude)),(row['lat'],row['lng'])), axis=1)


In [400]:
nearby_atm_only.head()

,name,categories,lat,lng,formattedAddress,distance
0,SABB ATM (صراف ساب),ATM,24.804728,46.749004,"['Dammam Road', 'الرياض', 'المملكة العربية الس...",2.165193
1,البنك الاهلي,ATM,24.824568,46.772067,"['المونسيه (شارع الصحابه)', 'الرياض 13253', 'ا...",4.139628
2,Alinma Bank (مصرف الإنماء),ATM,24.846634,46.782362,"['الرياض', 'المملكة العربية السعودية']",6.081298
3,SABB Bank ATM (بنك ساب),Bank,24.791585,46.729553,"['Dammam Road (Al Hamra)', 'الرياض', 'المملكة ...",2.792486
4,Alinma Bank (مصرف الإنماء),ATM,24.765788,46.706653,"['Al Imam Saud Bin Abdulaziz Rd', 'الرياض', 'ا...",6.202366


I limited the distance to '2.165200' because this is how far are the borders of the neiborhood from the center.

In [405]:
nearby_atm_only[nearby_atm_only['distance']<=2.165200] 

,name,categories,lat,lng,formattedAddress,distance
0,SABB ATM (صراف ساب),ATM,24.804728,46.749004,"['Dammam Road', 'الرياض', 'المملكة العربية الس...",2.165193
11,BANK ALJAZIRA (بنك الجزيرة),ATM,24.809890,46.734824,"['الرياض', 'المملكة العربية السعودية']",0.798723
12,ATM ALRJHI,Bank,24.831233,46.730631,"['طريق الثمامة', 'المملكة العربية السعودية']",1.632928


In [406]:
nearby_atm_only[nearby_atm_only['distance']<=2.165200].to_csv('ATM.csv', sep ='\t')